# [Lv4-Day1-Lab3] The Conductor: Manually Orchestrating a Multi-Step Agent

### 실습 목표
이전 실습에서 우리는 LLM이 단일 Tool을 사용하는 법(Function Calling)을 배웠습니다. 하지만 실무의 복잡한 문제는 여러 단계의 작업과 여러 번의 Tool 사용을 요구합니다. 이번 시간에는 **Multi-Agent 프레임워크 없이**, 오직 `Function Calling`과 순수 Python 코드(`for` 루프 등)만을 사용하여 여러 단계의 리서치 작업을 **'수동으로 지휘(Orchestrate)'** 해봅니다. 개발자가 직접 '프로젝트 매니저'이자 '지휘자(Conductor)'가 되어보는 것입니다.

1.  **Multi-Step 워크플로우 설계:** '계획 수립' -> '반복 실행' -> '결과 종합'의 3단계 워크플로우를 Python 함수로 명확히 분리하여 설계합니다.
2.  **수동 상태 관리:** 각 단계의 결과물(계획, 검색 데이터 등)을 Python 변수에 직접 저장하고 다음 단계로 전달하며, 수동 상태 관리의 복잡성을 체험합니다.
3.  **CrewAI의 필요성 체감:** 이 실습은 의도적으로 '성공'하도록 설계되었습니다. 하지만 이 성공 뒤에 숨겨진 번거롭고 복잡한 '지휘' 과정을 직접 경험함으로써, **"왜 우리는 CrewAI와 같은 자동화된 프레임워크가 필요한가?"** 에 대한 가장 강력하고 명확한 동기를 얻게 될 것입니다.

### 0. 사전 준비: 라이브러리 설치 및 API 키 설정
이전 실습(`Function Calling & Structured Output.ipynb`)에서 사용했던 모든 라이브러리와 API 키가 필요합니다. 새로운 런타임 환경을 가정하고 다시 설치 및 설정합니다.

In [1]:
# !pip install google-generativeai langchain_google_genai "pydantic>=2.0" instructor langchain-community tavily-python jsonref -q

#### API 키 발급 및 설정 안내 (모두 무료)

1.  **TAVILY API Key:**
    - 검색기능을 제공하는 API 입니다.
    - `https://www.tavily.com/` 사이트에 접속하여 가입합니다.
    - 로그인 후 대시보드에서 API Key를 확인하고 복사합니다. (무료 플랜으로 실습에 충분한 쿼리가 제공됩니다.)


In [2]:
import os
from getpass import getpass

# API 키 설정
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Google AI Studio API 키를 입력하세요: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass("TAVILY_API_KEY 를 입력하세요: ")

print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

Google AI Studio API 키를 입력하세요: ········
TAVILY_API_KEY 를 입력하세요: ········
✅ 모든 API 키가 성공적으로 설정되었습니다.


### 1단계: '계획 수립' 모듈 구현
첫 번째 LLM의 역할은 '계획가(Planner)'입니다. 사용자의 큰 주제를 입력받아, 이를 해결하기 위한 구체적인 세부 질문 리스트를 생성합니다. 이전 실습에서 배운 `instructor`와 `Pydantic`을 사용하여 LLM의 출력을 안정적인 Python 리스트(`List[str]`)로 강제합니다.

In [3]:
import instructor
from pydantic import BaseModel, Field
from typing import List


# 1. LLM으로부터 받을 데이터 구조(설계도) 정의
class ResearchPlan(BaseModel):
    questions: List[str] = Field(description="주어진 주제를 해결하기 위한, 서로 다른 관점의 구체적인 세부 질문 3개")


# 2. Instructor 클라이언트 생성
client = instructor.from_provider(
    "google/gemini-2.5-pro",
    mode=instructor.Mode.GEMINI_JSON,
)


# 3. '계획 수립' 함수 정의
def create_research_plan(topic: str) -> List[str]:
    """LLM을 사용하여 리서치 계획(세부 질문 리스트)을 생성합니다."""
    print(f"--- 🧠 1단계: '{topic}'에 대한 리서치 계획 수립 중... ---")
    plan = client.messages.create(
        messages=[
            {
                "role": "user",
                "content": f"'{topic}'에 대한 기술 보고서를 작성하려고 합니다. 보고서의 목차가 될 3개의 핵심 세부 질문을 생성해주세요.",
            }
        ],
        response_model=ResearchPlan,
    )
    print("✅ 계획 수립 완료!")
    return plan.questions


# 테스트
test_topic = "CrewAI와 LangGraph의 차이점"
research_plan = create_research_plan(test_topic)
print("\n[생성된 리서치 계획]")
for i, q in enumerate(research_plan, 1):
    print(f"{i}. {q}")

--- 🧠 1단계: 'CrewAI와 LangGraph의 차이점'에 대한 리서치 계획 수립 중... ---
✅ 계획 수립 완료!

[생성된 리서치 계획]
1. CrewAI는 역할 기반 에이전트 협업을, LangGraph는 상태 기반 그래프 아키텍처를 중심으로 설계되었습니다. 이 두 프레임워크의 핵심 아키텍처와 추상화 모델은 어떻게 다르며, 이것이 개발자의 에이전트 워크플로우 설계 방식에 어떤 영향을 미칩니까?
2. LangGraph는 상태(State)를 명시적으로 관리하고 순환(Cycle)을 허용하여 복잡한 제어 흐름을 구현하는 데 강점이 있습니다. 반면 CrewAI는 태스크(Task) 기반의 순차적 또는 계층적 프로세스를 중심으로 작동하는데, 두 프레임워크의 상태 관리 메커니즘과 제어 흐름 설계 방식의 근본적인 차이점은 무엇입니까?
3. 선형적인 콘텐츠 생성 작업과 같이 명확한 시작과 끝이 있는 프로젝트와, 사용자의 피드백을 반복적으로 반영하며 결과를 개선해야 하는 복잡한 분석 작업이 있다면, 각각의 시나리오에서 CrewAI와 LangGraph의 기술적 장단점은 어떻게 발휘됩니까?


### 2단계: '반복 실행' 모듈 구현
이제 우리가 '지휘자'가 되어, 1단계에서 생성된 질문 리스트를 하나씩 순회하며 웹 검색을 수행합니다. 이 과정은 CrewAI의 `Sequential Process`를 우리가 Python의 `for` 루프로 직접 흉내 내는 것입니다.

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

# '리서처' 역할을 할 Tool 정의
web_search_tool = TavilySearchResults(max_results=3)


def execute_research(questions: List[str]) -> List[dict]:
    """질문 리스트를 받아 각 질문에 대한 웹 검색을 수행하고 결과를 수집합니다."""
    print(f"\n--- 🛠️ 2단계: 총 {len(questions)}개의 질문에 대한 반복 리서치 실행 중... ---")
    all_research_data = []
    for i, question in enumerate(questions, 1):
        print(f"  ({i}/{len(questions)}) '{question}' 검색 중...")
        try:
            search_result = web_search_tool.invoke(question)
            all_research_data.append({"question": question, "result": search_result})
        except Exception as e:
            print(f"  🔴 검색 실패: {e}")
            all_research_data.append({"question": question, "result": "검색 중 오류가 발생했습니다."})
    print("✅ 모든 리서치 실행 완료!")
    return all_research_data


# 테스트
research_results = execute_research(research_plan)
print("\n[첫 번째 리서치 결과 (일부)]")
print(str(research_results[0])[:500] + "...")


--- 🛠️ 2단계: 총 3개의 질문에 대한 반복 리서치 실행 중... ---
  (1/3) 'CrewAI는 역할 기반 에이전트 협업을, LangGraph는 상태 기반 그래프 아키텍처를 중심으로 설계되었습니다. 이 두 프레임워크의 핵심 아키텍처와 추상화 모델은 어떻게 다르며, 이것이 개발자의 에이전트 워크플로우 설계 방식에 어떤 영향을 미칩니까?' 검색 중...
  (2/3) 'LangGraph는 상태(State)를 명시적으로 관리하고 순환(Cycle)을 허용하여 복잡한 제어 흐름을 구현하는 데 강점이 있습니다. 반면 CrewAI는 태스크(Task) 기반의 순차적 또는 계층적 프로세스를 중심으로 작동하는데, 두 프레임워크의 상태 관리 메커니즘과 제어 흐름 설계 방식의 근본적인 차이점은 무엇입니까?' 검색 중...
  (3/3) '선형적인 콘텐츠 생성 작업과 같이 명확한 시작과 끝이 있는 프로젝트와, 사용자의 피드백을 반복적으로 반영하며 결과를 개선해야 하는 복잡한 분석 작업이 있다면, 각각의 시나리오에서 CrewAI와 LangGraph의 기술적 장단점은 어떻게 발휘됩니까?' 검색 중...
✅ 모든 리서치 실행 완료!

[첫 번째 리서치 결과 (일부)]
{'question': 'CrewAI는 역할 기반 에이전트 협업을, LangGraph는 상태 기반 그래프 아키텍처를 중심으로 설계되었습니다. 이 두 프레임워크의 핵심 아키텍처와 추상화 모델은 어떻게 다르며, 이것이 개발자의 에이전트 워크플로우 설계 방식에 어떤 영향을 미칩니까?', 'result': [{'title': '초보자를 위한 2025년 AI 에이전트 프레임워크 가이드: LangGraph ...', 'url': 'https://cyan91.tistory.com/entry/%EC%B4%88%EB%B3%B4%EC%9E%90%EB%A5%BC-%EC%9C%84%ED%95%9C-2025%EB%85%84-AI-%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8-%ED%94%84%EB%A0%88%EC%

### 3단계: '결과 종합' 모듈 구현
마지막으로, '보고서 작성가' 역할을 할 LLM을 호출합니다. 2단계에서 수집한 모든 리서치 결과를 하나의 거대한 컨텍스트로 묶어 LLM에게 전달하고, 최종 보고서 작성을 요청합니다.

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
import json

# '보고서 작성가' 역할을 할 일반 LLM 초기화
writer_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.1)


def synthesize_report(topic: str, research_data: List[dict]) -> str:
    """수집된 모든 리서치 데이터를 바탕으로 최종 보고서를 작성합니다."""
    print(f"\n--- 📝 3단계: 최종 보고서 종합 중... ---")

    # LLM에게 전달할 컨텍스트 생성
    context_string = ""
    for item in research_data:
        context_string += f"질문: {item['question']}\n"
        context_string += f"조사된 내용: {json.dumps(item['result'], ensure_ascii=False)}\n\n"

    prompt = f"""당신은 전문 기술 분석가입니다.
    다음은 '{topic}'이라는 주제에 대해 조사된 자료입니다.

    --- 조사 자료 ---
    {context_string}
    --- 자료 끝 ---

    위 자료만을 바탕으로, '{topic}'에 대한 상세하고 구조화된 기술 분석 보고서를 작성해주세요.
    보고서는 개요, 각 질문에 대한 상세 분석, 최종 결론의 순서로 구성되어야 합니다."""

    response = writer_llm.invoke(prompt)
    print("✅ 최종 보고서 작성 완료!")
    return response.content


# 테스트
final_report = synthesize_report(test_topic, research_results)
print("\n[최종 보고서 (일부)]")
print(final_report)


--- 📝 3단계: 최종 보고서 종합 중... ---
✅ 최종 보고서 작성 완료!

[최종 보고서 (일부)]
## CrewAI와 LangGraph의 차이점: 기술 분석 보고서

### **개요**

본 보고서는 AI 에이전트 개발 프레임워크인 CrewAI와 LangGraph의 핵심적인 차이점을 기술적 관점에서 심층 분석합니다. 제공된 조사 자료에 근거하여, 두 프레임워크는 다중 에이전트 시스템 구축이라는 공통된 목표를 가지지만, 근본적으로 다른 설계 철학과 아키텍처를 채택하고 있음을 확인했습니다.

**CrewAI**는 **역할 기반 협업(Role-based Collaboration)** 모델을 중심으로, 각 AI 에이전트에게 명확한 역할과 책임을 부여하여 인간 조직과 유사한 방식으로 작업을 수행하도록 설계되었습니다. 이는 개발자가 '누가 무엇을 할 것인가'에 집중하게 하여 직관적이고 빠른 개발을 가능하게 합니다.

반면, **LangGraph**는 **상태 기반 그래프(State-based Graph)** 아키텍처를 기반으로 합니다. 워크플로우를 노드(Node)와 엣지(Edge)로 구성된 그래프로 명시적으로 정의하며, 각 단계의 상태(State)를 체계적으로 관리합니다. 이를 통해 개발자는 '작업이 어떤 조건과 순서로 흘러갈 것인가'를 세밀하게 제어할 수 있어, 복잡하고 비선형적인 프로세스 구현에 강점을 보입니다.

본 보고서는 두 프레임워크의 아키텍처, 상태 관리 및 제어 흐름, 그리고 실제 적용 시나리오에서의 장단점을 상세히 비교 분석하여 개발자가 프로젝트의 특성에 맞는 최적의 프레임워크를 선택할 수 있도록 돕는 것을 목표로 합니다.

---

### **상세 분석**

#### **1. 핵심 아키텍처와 추상화 모델의 차이 및 개발 방식에 미치는 영향**

두 프레임워크의 가장 근본적인 차이는 아키텍처와 추상화 모델에서 비롯되며, 이는 개발자의 에이전트 워크플로우 설계 방식에 직접적인 영향을 미칩니다.

*   **CrewAI: 역할 중심의 고수준 추상

### 4. 최종 분석: 무엇이 번거로웠고, 무엇을 자동화할 수 있을까?
**성공적으로 Multi-Step 리서치 보고서가 완성되었습니다!** 하지만 그 과정을 되돌아봅시다. 우리는 '지휘자'로서 다음과 같은 번거로운 작업들을 **직접** 수행해야 했습니다.

1.  **수동 상태 관리 (Manual State Management):**
    - `research_plan`이라는 변수를 만들어 1단계의 결과를 저장했습니다.
    - 이 변수를 `execute_research` 함수에 **직접 전달**해야 했습니다.
    - `research_results`라는 변수를 만들어 2단계의 결과를 저장하고, 이를 다시 `synthesize_report` 함수에 **직접 전달**해야 했습니다.
    - 만약 단계가 10개라면, 10개의 변수를 만들고 수동으로 연결해야 했을 것입니다.

2.  **명시적인 제어 흐름 (Explicit Control Flow):**
    - 2단계의 반복적인 리서치 작업을 위해 **`for` 루프**를 직접 작성해야 했습니다.
    - 각 단계(함수)를 **올바른 순서대로 호출**하는 `main` 로직을 우리가 직접 관리해야 했습니다.
